<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_NLP_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Natural Language Processing

In [37]:
import tensorflow as tf

import pandas as pd

import random

from tensorflow.keras.layers import Embedding, TextVectorization

from sklearn.model_selection import train_test_split

## Constantes

In [2]:
SEMENTE = 2008193

## Funções

In [3]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

--2023-01-27 21:38:57--  https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5196 (5.1K) [text/plain]
Saving to: ‘funcoes.py’

funcoes.py          100%[===================>]   5.07K  --.-KB/s    in 0s      

2023-01-27 21:38:58 (46.3 MB/s) - ‘funcoes.py’ saved [5196/5196]



In [4]:
from funcoes import criar_callback_tensorboard, grafico_historico_por_iteracao, unzip_data

## Dados

Dados originalmente da competição **Natural Language Processing with Disaster Tweets** no Kaggle. (Em andamento em 27.01.2023.)

https://www.kaggle.com/c/nlp-getting-started

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data('nlp_getting_started.zip')

--2023-01-27 21:38:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2023-01-27 21:38:59 (73.6 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
df_treino = pd.read_csv('train.csv')
df_teste = pd.read_csv('test.csv')

# Embaralhar dados de treino (frac=1 = 100% dos dados)
df_treino = df_treino.sample(frac=1, random_state=SEMENTE)

In [7]:
df_treino.head()

,id,keyword,location,text,target
2299,3299,demolish,NYHC,If you think going to demolish Drake's house o...,1
3704,5269,fear,Carregado,I didn`t want to hurt you but the fear drove m...,0
3468,4961,explosion,New York,New Explosion-proof Tempered Glass Screen Prot...,0
6653,9533,terrorist,????? ???? ????,#UdhampurAgain 2 terrorist shot dead.. #Udhampur,1
3634,5183,fatalities,jersey,mortal kombat fatalities are so gross but inte...,0


In [8]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 2299 to 4895
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 356.9+ KB


In [9]:
df_treino['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

### Separar dados para validação

In [10]:
X = df_treino['text']
y = df_treino['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEMENTE)

## Converter texto em números

### Vetorização (Vectorization)

In [34]:
# # Valores-padrão para TextVectorization
# vetorizador = TextVectorization(max_tokens=None,                            # Máximo de tokens (palavras) no dicionário. (Palavras acima do limite são classificadas como OOV.)
#                                 standardize='lower_and_strip_punctuation',  # Texto em caixa baixa e retira caracteres especiais.
#                                 split='whitespace',                         # Separador de tokens.
#                                 ngrams=None,                                # Tokens por grupo. (None = não agrupar tokens.)
#                                 output_mode='int',                          # Como mapear tokens para números.
#                                 output_sequence_length=None)                # Tamanho da sequencia de tokens (texto), completado ou truncado, se necessário. (Válido apenas para 'int'.)

In [12]:
limite_dicionario = 10000
media_palavras_por_texto = round(sum([len(texto.split()) for texto in X_train]) / len(X_train))

vetorizador = TextVectorization(max_tokens=limite_dicionario,
                                output_mode='int',
                                output_sequence_length=media_palavras_por_texto)

In [13]:
vetorizador.adapt(X_train)

In [26]:
vetorizador(['At least six children were shot dead between January 2022 and 2023.'])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  17,  466, 3430,  658,   70,  644,  119, 1323,    1,    1,    7,
           1,    0,    0,    0]])>

In [27]:
texto = random.choice(X_train)
print(texto)

vetorizador([texto])

Maryland mansion fire that killed 6 caused by damaged plug under Christmas tree report says - Into the flames: Firefighter's bravery...


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[2390, 2393,   40,   16,  121,  478,  583,   18, 1421, 2033,  211,
        1908,  985,  338,  286]])>

In [32]:
vocabulario = vetorizador.get_vocabulary()

In [35]:
# [UNK] = OOV (out of vocabulary)
len(vocabulario), vocabulario[:5], vocabulario[-5:]

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pacic', 'p45perez', 'oyster', 'oworoshoki', 'ownership'])

### Incorporação (Embedding)

In [39]:
incorporador = Embedding(input_dim=limite_dicionario,            # Tamanho do vocabulário de entrada.
                         output_dim=128,                         # Dimensão do vetor de saída. (Geralmente um múltiplo de 8.)
                         input_length=media_palavras_por_texto)  # Tamanho da sequencia de tokens de entrada.

In [50]:
texto = random.choice(X_train)
print(texto)

incorporador(vetorizador([texto]))

Economic Collapse Investing: Specific actions and strategies to securing lasting wealth from the financial blowout. http://t.co/JZwRisXEPF


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03469752,  0.00746028, -0.04331923, ..., -0.04789924,
          0.01503421,  0.04148102],
        [-0.03190859, -0.02742977,  0.01179903, ...,  0.02182505,
         -0.04445665, -0.00473956],
        [-0.02234453,  0.034354  , -0.00103768, ...,  0.02776172,
         -0.02190968,  0.02673883],
        ...,
        [ 0.03364093,  0.01475663, -0.00127186, ...,  0.01705715,
          0.00802834,  0.00407535],
        [-0.040479  ,  0.00240358,  0.03241447, ..., -0.02179515,
         -0.0095867 ,  0.0236893 ],
        [-0.03934635, -0.03947687,  0.00204285, ..., -0.03358956,
          0.04578893, -0.03839893]]], dtype=float32)>